In [1]:
%matplotlib inline

from utils import *

In [2]:
pop_data     = pandas.read_csv(
    locate('data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv')).groupby(
['NTA Code',
'NTA Name']
).apply(lambda x :
       pandas.Series(
       {'pop_2000':x.query('Year==2000').Population.mean(),
       'pop_2010':x.query('Year==2010').Population.mean()})).reset_index()
    
print(pop_data.shape)
pop_data.head()


(195, 4)


,NTA Code,NTA Name,pop_2000,pop_2010
0,BK09,Brooklyn Heights-Cobble Hill,22548.0,22887.0
1,BK17,Sheepshead Bay-Gerritsen Beach-Manhattn Bch,64596.0,64518.0
2,BK19,Brighton Beach,35244.0,35547.0
3,BK21,Seagate-Coney Island,34267.0,31965.0
4,BK23,West Brighton,17370.0,17750.0


In [3]:

shapes = geopandas.read_file(
    locate('data/NTA map.geojson'))
print(shapes.shape)

shapes.head()

(195, 8)


,ntacode,shape_area,ntaname,shape_leng,boroname,borocode,countyfips,geometry
0,BK43,35799637.8103,Midwood,27996.5912736,Brooklyn,3,047,(POLYGON ((-73.94732672160586 40.6291665672094...
1,BK75,32629833.1149,Bedford,29992.9191744,Brooklyn,3,047,(POLYGON ((-73.94193078816201 40.7007252346955...
2,BX40,6307283.62202,Fordham South,15878.2729212,Bronx,2,005,(POLYGON ((-73.89138023380268 40.8617005882630...
3,BK88,54005019.2286,Borough Park,39247.227722,Brooklyn,3,047,(POLYGON ((-73.9760493559142 40.63127590564666...
4,BK96,32706946.4035,Rugby-Remsen Village,30957.8533949,Brooklyn,3,047,(POLYGON ((-73.90855790522774 40.6520959377945...


In [4]:
roads = geopandas.read_file(
    locate('data/road.geojson')
)[[u'physicalid',u'geometry']]
print(roads.shape)

roads.head()

(117191, 2)


,physicalid,geometry
0,37785,LINESTRING (-73.98027180261785 40.685958254199...
1,7141,LINESTRING (-73.84554250435114 40.739657802135...
2,38225,LINESTRING (-73.96582204725706 40.688209422824...
3,166986,LINESTRING (-73.85196038837651 40.574280057286...
4,97055,LINESTRING (-73.86924838521583 40.740570212006...


In [5]:
popshapes = geopandas.GeoDataFrame(shapes.merge(
    pop_data,
    how ='left',
    right_on = 'NTA Code',
    left_on='ntacode'
).assign(
evpop = lambda DF : (
    DF.pop_2010 -DF.pop_2000
)/DF.pop_2000
).fillna(0)[
    ['ntacode','ntaname','shape_area','shape_leng','pop_2000','pop_2010','evpop','geometry']
])
print(popshapes.shape)
popshapes.head()

(195, 8)


,ntacode,ntaname,shape_area,shape_leng,pop_2000,pop_2010,evpop,geometry
0,BK43,Midwood,35799637.8103,27996.5912736,55440.0,52835.0,-0.046988,(POLYGON ((-73.94732672160586 40.6291665672094...
1,BK75,Bedford,32629833.1149,29992.9191744,59227.0,70713.0,0.193932,(POLYGON ((-73.94193078816201 40.7007252346955...
2,BX40,Fordham South,6307283.62202,15878.2729212,26880.0,28262.0,0.051414,(POLYGON ((-73.89138023380268 40.8617005882630...
3,BK88,Borough Park,54005019.2286,39247.227722,101055.0,106357.0,0.052466,(POLYGON ((-73.9760493559142 40.63127590564666...
4,BK96,Rugby-Remsen Village,32706946.4035,30957.8533949,57934.0,55326.0,-0.045017,(POLYGON ((-73.90855790522774 40.6520959377945...


In [6]:
popshapes.tail()

,ntacode,ntaname,shape_area,shape_leng,pop_2000,pop_2010,evpop,geometry
190,BX63,West Concourse,19369542.2241,28550.7003701,39557.0,39282.0,-0.006952,(POLYGON ((-73.91191812320277 40.8432578866849...
191,QN17,Forest Hills,57828447.5721,36183.6078275,85046.0,83728.0,-0.015497,(POLYGON ((-73.83103141767994 40.7143367252613...
192,QN99,park-cemetery-etc-Queens,311684971.61,490146.60402,1785.0,436.0,-0.755742,(POLYGON ((-73.86522555434871 40.5704584724939...
193,QN03,Springfield Gardens South-Brookville,43263751.4662,45266.2678751,18950.0,20132.0,0.062375,(POLYGON ((-73.75763129618157 40.6664511566070...
194,QN98,Airport,227761739.733,134833.935261,0.0,0.0,0.000000,(POLYGON ((-73.74712154262478 40.6371622757079...


In [7]:
ids         = RightSpatialJoin(
    L_bounds=popshapes.geometry.apply(lambda x : x.bounds),
    R_bounds=roads.geometry.apply(lambda x : x.bounds))

In [8]:
popshapes[
    'nb_roads'
         ] = ids.apply(lambda x : roads.loc[x].physicalid.unique().shape[0])
del(roads)
del(ids)
del(shapes)
del(pop_data)

In [9]:
popshapes

,ntacode,ntaname,shape_area,shape_leng,pop_2000,pop_2010,evpop,geometry,nb_roads
0,BK43,Midwood,35799637.8103,27996.5912736,55440.0,52835.0,-0.046988,(POLYGON ((-73.94732672160586 40.6291665672094...,692
1,BK75,Bedford,32629833.1149,29992.9191744,59227.0,70713.0,0.193932,(POLYGON ((-73.94193078816201 40.7007252346955...,770
2,BX40,Fordham South,6307283.62202,15878.2729212,26880.0,28262.0,0.051414,(POLYGON ((-73.89138023380268 40.8617005882630...,346
3,BK88,Borough Park,54005019.2286,39247.227722,101055.0,106357.0,0.052466,(POLYGON ((-73.9760493559142 40.63127590564666...,1447
4,BK96,Rugby-Remsen Village,32706946.4035,30957.8533949,57934.0,55326.0,-0.045017,(POLYGON ((-73.90855790522774 40.6520959377945...,1223
5,QN52,East Flushing,29454379.9663,25843.3649557,26516.0,26982.0,0.017574,(POLYGON ((-73.79493246234003 40.7577980300745...,538
6,QN53,Woodhaven,37001316.6276,28070.5855484,54149.0,56674.0,0.046631,(POLYGON ((-73.84913837242911 40.6982827412019...,1091
7,BK44,Madison,27379161.9978,26237.257341,39076.0,38917.0,-0.004069,(POLYGON ((-73.93753749374048 40.6085573902575...,678
8,QN48,Auburndale,34164217.5747,32446.878673,20201.0,19996.0,-0.010148,(POLYGON ((-73.77573836927105 40.7433256471640...,1489
9,BX44,Williamsbridge-Olinville,36273563.8081,27350.9739448,57420.0,61321.0,0.067938,(POLYGON ((-73.84704957988549 40.8876718158794...,884


In [10]:
class GetFeatures:
    def __init__(
        self,
        filepath,
        popshapes                 = popshapes,
        readargs                  = readargs,
        writeargs                 = writeargs):
        self.writeargs            = writeargs
        self.filepath             = filepath
        self.df                   = pandas.read_hdf(filepath,**readargs).dropna()
        self.df                   = self.df.assign(
        geometry                  = [shapely.geometry.LineString([x,y])
                                     for x,y in
                                     tqdm.tqdm(zip(self.df[
                                         ['pickup_longitude','pickup_latitude']
                                     ].values,
                                                   self.df[
                                                       ['dropoff_longitude','dropoff_latitude']
                                                   ].values))]
        ).astype(
        {'pickup_datetime':'datetime64[ns]'}).assign(
                dist               = lambda DF : geopandas.GeoSeries(DF.geometry).length,
                dayofweek          = lambda DF : DF.pickup_datetime.dt.dayofweek,
                dayofyear          = lambda DF : DF.pickup_datetime.dt.dayofyear,
                hour               = lambda DF : DF.pickup_datetime.dt.hour,
                month              = lambda DF : DF.pickup_datetime.dt.month,
                 weekofyear        = lambda DF : DF.pickup_datetime.dt.weekofyear,
                        year       = lambda DF : DF.pickup_datetime.dt.year).reset_index(drop=True)
        self.numcols               = ['pop_2010','nb_roads']
        self.popshapes             = popshapes
    def matchGeoShapes(self):
            ids2                   = RightSpatialJoin(
            L_bounds               = self.df.geometry.apply(lambda x : x.bounds),
            R_bounds               = self.popshapes.geometry.apply(lambda x : x.bounds))
            result                 = self.popshapes[self.numcols].loc[[ h for x in ids2 for h in x]]
            result['row_id']       = [index
                                      for item in ids2.iteritems() 
                                      for index
                                      in [item[0]]*len(item[1])]
            self.result            = pandas.DataFrame({'row_id': [x for x in range(len(ids2))]}
            ).merge(result,how     ='left',on='row_id').fillna(0
                                                         ).pipe(
                lambda R : pandas.DataFrame(
                 {'pop_2010_mean'  : R.groupby('row_id').pop_2010.mean(),
                  'nb_roads_mean'  : R.groupby('row_id').nb_roads.mean(),
                  'pop_2010_sum'   : R.groupby('row_id').pop_2010.sum(),
                 'nb_roads_sum'    : R.groupby('row_id').nb_roads.sum(),
                 'pop_2010_min'    : R.groupby('row_id').pop_2010.min(),
                 'nb_roads_min'    : R.groupby('row_id').nb_roads.min(),
                 'pop_2010_max'    : R.groupby('row_id').pop_2010.max(),
                 'nb_roads_max'    : R.groupby('row_id').nb_roads.max()}))
            pandas.concat(
                    [self.df,self.result]
                    ,axis=1).to_hdf(self.filepath,**self.writeargs)
            

In [ ]:





def main(max_workers,Task):
    with Cfut.ProcessPoolExecutor(max_workers=max_workers) as executor:
        executor.map(Task, range(max_workers))

if __name__ == '__main__'           :
    max_workers                     = 4
    trainfiles                      = pandas.np.array_split(H5('train'),max_workers)
    def Task(n,tf=trainfiles)       :
        traineloop                  = tf[n]
        map(lambda f                : GetFeatures(f).matchGeoShapes(),traineloop)
    main(max_workers,Task=Task)

In [51]:
pandas.np.array_split(H5('train'),1)

[array(['/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/20.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/14.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/9.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/10.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/24.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/8.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/11.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/25.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/21.h5',
        '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/15.h5',
        '/Users/akamara/Downloads/new-york-city-taxi